In [13]:
# Import necessary libraries
import pandas as pd
import scrapy
from scrapy import Selector
import requests
from bs4 import BeautifulSoup
import re
from datetime import date

In [14]:
# Get a list of links to products of particular category (smartphones)
fulllinks = []
for i in range (1, 19):
    url = 'https://rozetka.com.ua/ua/mobile-phones/c80003/page={};preset=smartfon/'.format(i)
    html = requests.get( url ).content
    sel = Selector( text=html )
    for link in sel.xpath('//a[@class="goods-tile__heading"]/@href').getall():
        fulllinks.append(link)

In [15]:
# Count number of links retrived
len(pd.DataFrame(fulllinks)[0].unique())

987

In [16]:
# Iterate through list of links to products, parse html to retrieve needed data and save it as list of lists  
dt = []
for i in fulllinks:
    html = requests.get( i +'/comments/' ).content
    html = html.replace(b'<br>', b'')
    html = html.replace(b'<br/>', b'')
    html = html.replace(b'\n', b'')
    sel = Selector( text=html )
    
    link_id = sel.xpath('//a[@class="button button_type_link comment__link"]/@href').getall()
    link_id = set(link_id)
    ids=[]
    for i in link_id:
        _, j = i.split('=')
        ids.append(j)
    #link = re.search('(.*)/comments/*', i).group(1)
        
    title = sel.xpath('//h1[@class="product__title"]/text()').extract_first()
    price = sel.xpath('//script [@type="application/ld+json"][@data-seo="Product"]/text()').extract_first()
    price = re.search('"price":"(.*)","priceCurrency"', price).group(1)
    
    for id_number in ids:
        rewiew_date = sel.xpath('//a[@class="button button_type_link comment__link" and contains(@href,"{}")]//preceding::time[@class="comment__date"]/text()'.format(id_number)).getall()[-1]
        rewiew_text_in = sel.xpath('//div[@class="comment"]//a[@class="button button_type_link comment__link" and contains(@href,"{}")]/following::rz-comment-rating/following::*[1]'.format(id_number)).extract_first()

        rating_numbers=[]
        for star_number in range (5):
            path = str(sel.xpath('//rz-rating-stars[@class="comment__rating-stars"][@id="{}"]/*/li[@class="rating-stars__item"][@data-index-number="{}"]/*/path'.format(id_number, star_number)).extract_first())
            if path !="None":
                _, j = path.split('#')
                j, _ = j.split(')')
                rating_numbers.append(int(j))
        rewiew_feedback_usefull = sel.xpath('//a[@class="button button_type_link comment__link" and contains(@href,"{}")]//following::button[@aria-label="Полезный отзыв"]/text()'.format(id_number)).extract_first()
        rewiew_feedback_notusefull = sel.xpath('//a[@class="button button_type_link comment__link" and contains(@href,"{}")]//following::button[@aria-label="Неполезный отзыв"]/text()'.format(id_number)).extract_first()
        if rewiew_text_in.startswith("<p class"):
            rewiew_text = re.search('<p class="comment__text">(.*)</p>', rewiew_text_in).group(1)
            product_advatages_disadvantages = sel.xpath('//a[@class="button button_type_link comment__link" and contains(@href,"{}")]/following::p[@class="comment__text"]/following::*[1]'.format(id_number)).extract_first()
        else:
            rewiew_text = "None"
            product_advatages_disadvantages = rewiew_text_in
        
        if "Переваги" in product_advatages_disadvantages:
            product_advatages = re.search('Переваги: </dt><dd>(.*)</dd>', product_advatages_disadvantages).group(1)
        else:
            product_advatages = 'None'
        if "Недоліки" in product_advatages_disadvantages:
            product_disadvantages = re.search('Недоліки: </dt><dd>(.*)</dd>', product_advatages_disadvantages).group(1)
        else:
            product_disadvantages = 'None'
        
        dt.append([i, title, price, id_number, rewiew_date, rewiew_text, sum(rating_numbers), rewiew_feedback_usefull, rewiew_feedback_notusefull, product_advatages_disadvantages, product_advatages, product_disadvantages])    

In [17]:
# Transform retrieved  data to DataFrame
data = pd.DataFrame(dt, columns = ['product_link', 'product_title', 'product_price','review_id', 'review_date', 'review_text', 'review_rating', 'thumbs_up', 'thumbs_down', 'product_advatages_disadvantages', 'product_advatages', 'product_disadvantages'])

In [18]:
# View characteristics of retrieved data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6154 entries, 0 to 6153
Data columns (total 12 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   product_link                     6154 non-null   object
 1   product_title                    6154 non-null   object
 2   product_price                    6154 non-null   object
 3   review_id                        6154 non-null   object
 4   review_date                      6154 non-null   object
 5   review_text                      6154 non-null   object
 6   review_rating                    6154 non-null   int64 
 7   thumbs_up                        6154 non-null   object
 8   thumbs_down                      6154 non-null   object
 9   product_advatages_disadvantages  6154 non-null   object
 10  product_advatages                6154 non-null   object
 11  product_disadvantages            6154 non-null   object
dtypes: int64(1), object(11)
memory usa

In [19]:
# View first 5 rows of retrieved data
data.head(5)

,product_link,product_title,product_price,review_id,review_date,review_text,review_rating,thumbs_up,thumbs_down,product_advatages_disadvantages,product_advatages,product_disadvantages
0,https://rozetka.com.ua/ua/samsung_galaxy_s21_p...,Мобільний телефон Samsung Galaxy S21 Ultra 12...,39999.00,49470910,02 березня 2021,Мне пришлось столкнуться с этим флагманом Sams...,3,0,10,"<dl class=""comment__essentials""><!----><div cl...","Графика экрана, производительность, качество с...","Отсутствие стереозвучания, звук сырой; Отсутви..."
1,https://rozetka.com.ua/ua/samsung_galaxy_s21_p...,Мобільний телефон Samsung Galaxy S21 Ultra 12...,39999.00,49481282,03 березня 2021,Представляємо новий сміливий дизайн камери окр...,4,1,6,"<dl class=""comment__essentials""><!----><div cl...",Відтінки Ultra,None
2,https://rozetka.com.ua/ua/samsung_galaxy_s21_p...,Мобільний телефон Samsung Galaxy S21 Ultra 12...,39999.00,49285480,12 лютого 2021,Очень нравиться samsung тем что оставили изогн...,5,1,3,"<rz-comment-conformity _nghost-sc93=""""><!---->...",None,None
3,https://rozetka.com.ua/ua/samsung_galaxy_s21_p...,Мобільний телефон Samsung Galaxy S21 Ultra 12...,39999.00,49505480,05 березня 2021,Добрый день!Где можно увидеть результаты розыг...,5,2,1,"<rz-comment-conformity _nghost-sc93=""""><!---->...",None,None
4,https://rozetka.com.ua/ua/samsung_galaxy_s21_p...,Мобільний телефон Samsung Galaxy S21 Ultra 12...,39999.00,49517759,сьогодні,Вероятность бракованного товара очень высокая!...,1,0,3,"<dl class=""comment__essentials""><!----><div cl...","Нет</dd></div><!----><div class=""comment__esse...",Вся техніка самсунг


In [20]:
# Write created DaraFrame tit h csv file
data.to_csv('smartphone_reviews_{}.csv'.format(today), index=False)

In [21]:
# View “prettified” html of product link with Beautiful soup (useful during debagging of html parsing)
text = sel.xpath('//*').getall()
listToStr = ' '.join(map(str, text))
soup = BeautifulSoup(listToStr)
soup =soup.prettify() 
print(soup)

<html lang="ru">
 <head>
  <title>
   ROZETKA  | Відгуки про Мобільний телефон ZTE Blade A5 2020 2/32GB Black: обговорення, фото, відео. Купити Мобільний телефон ZTE Blade A5 2020 2/32GB Black в Києві
  </title>
  <meta content=" Відгуки про Мобільний телефон ZTE Blade A5 2020 2/32GB Black купити на ROZETKA. Оперативна доставка ✈ Гарантія якості ☑ Найкраща ціна $" name="description"/>
  <meta content="відгуки про Мобільний телефон ZTE Blade A5 2020 2/32GB Black" name="keywords"/>
  <meta charset="utf-8"/>
  <script>
   dataLayer = [];
  </script>
  <script>
   const Cart = new Object({            purchases   : [],            setPurchases: function (purchases) {                this.purchases = purchases;            },            getPurchases: function () {                return this.purchases;            },        });
  </script>
  <script async="async" src="https://www.googletagservices.com/tag/js/gpt.js">
  </script>
  <script>
   var googletag = googletag || {};        googletag.cmd 